<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/NIS_11_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **НИС SENTI-FRAME**

In [0]:
!pip install pymorphy2[fast]
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 7.1MB 13.6MB/s 
     |████████████████████████████████| 378kB 48.7MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=843847 sha256=7617bbc71d74f71c475d150b4745c3b5b94fc86623475ed6d6b6037619e79078
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
#!rm collection.json.1 collection.json
#!ls
import gensim
from collections import Counter
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
%%capture
# at least two mutual negative attitudes
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/neg_a0_a1_mutual.json
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/neg_a0_a2_mutual.json
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/neg_a1_a2_mutual.json

# at least two mutual positive attitudes
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/pos_a0_a1_mutual.json
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/pos_a0_a2_mutual.json
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/pos_a1_a2_mutual.json

# at least two opposite attitudes (only a0/a1 show opposite attitudes toward each other)
!wget https://raw.githubusercontent.com/DmitryKutsev/NIS_SentiFrame/master/grouped_verb_lists/opp_a0_a1_candidates_l.json

In [0]:
fnames = '''
neg_a0_a1_mutual.json  opp_a0_a1_candidates_l.json  pos_a1_a2_mutual.json
neg_a0_a2_mutual.json  pos_a0_a1_mutual.json	    sample_data
neg_a1_a2_mutual.json  pos_a0_a2_mutual.json
'''.split()

## **Импорт с Rusvectores skipgram и fasttext моделей, обученных на корпусе Тайга.**

In [4]:
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
!wget http://vectors.nlpl.eu/repository/20/187.zip
!unzip 187.zip

--2020-03-11 15:23:17--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz’

news_mystem_skipgra 100%[===================>] 524.47M  19.8MB/s    in 28s     

2020-03-11 15:23:45 (19.0 MB/s) - ‘news_mystem_skipgram_1000_20_2015.bin.gz’ saved [549952184/549952184]

--2020-03-11 15:23:56--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  17.0MB/s    in 2m 39s 

In [0]:
collection_handler = open("collection.json", "r", encoding="utf-8")
collection = json.load(collection_handler, encoding="utf-8")
skipgram_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)
fasttext_model = gensim.models.KeyedVectors.load("model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Мультиязычный **BERT** для векторизации предложений

Check out [this issue](https://github.com/hanxiao/bert-as-service/issues/380) and "make sure Colab is using Tensorflow 1.x, because bert-serving-start doesn't currently work with TF 2.1 and nohup hides the output of the command failing"

In [0]:
# import tensorflow as tf
# print (tf.__version__)

In [0]:
%%capture
!pip install -U bert-serving-server[http]
!pip install bert-serving-client  # client, independent of `bert-serving-server`

In [0]:
%%capture
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip /content/multi_cased_L-12_H-768_A-12.zip

In [0]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.49155593,  0.08795979,  0.08263359, ...,  1.0980439 ,
         0.41126642, -0.25396958],
       [-0.07413451, -0.2278353 , -0.08978202, ...,  1.5093102 ,
         1.3512001 , -0.03158369],
       [-0.2657526 ,  0.1913553 , -0.3561356 , ...,  1.3962169 ,
         1.4187483 ,  0.12651931]], dtype=float32)

In [0]:
def a0_a1_clause_maker(verb, arg0, arg1, arg0_case='nomn', arg1_case='accs', cap=True):

  '''Генерирует простые предложения вида 'arg0 любит arg1'.

  Параметры:

  verb              глагол в любой форме;
  arg0_case='nomn'  тег падежа подлежащего;
  arg1_case='accs'  тег падежа дополнения;
  arg0='Маша'       подлежащее;
  arg1='Петя'       дополнение
  cap=True        капитализация аргументов-имен собственных.
  
  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.
  
  Example
  
  a0_a1_clause_maker('любить', 'Петя', 'Маша')

  'Петя любит Машу'
  '''

  inflected_arg0 = morph.parse(arg0)[0].inflect({'sing', 'nomn'}).word
  inflected_verb = morph.parse(verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  inflected_arg1 = morph.parse(arg1)[0].inflect({'sing', 'accs'}).word

  if (cap == True) and ('Name' in morph.parse(arg1)[0].tag):
    inflected_arg1 = inflected_arg1.capitalize()
  if (cap == True) and ('Name' in morph.parse(arg0)[0].tag):
    inflected_arg0 = inflected_arg0.capitalize()

  sentence = '{} {} {}'.format(inflected_arg0, inflected_verb, inflected_arg1)
  return sentence

# a0_a1_clause_maker('любить', 'Маша', 'Петя')
a0_a1_clause_maker('любить', 'Петя', 'Маша')


  

'Петя любит Машу'

In [0]:
# Пете нравится, что
def clause_wrapper(verb, arg0, arg1, 
                   main_subject=1, main_verb='нравиться',
                   arg0_case='nomn', arg1_case='accs', 
                   cap=True):

  '''Генерирует сложноподчинённые предложения вида 'arg нравится, что arg0 любит arg1'.

  Параметры:

  verb                    глагол-предикат подчинённой клаузы;
  arg0_case='nomn'        тег падежа подлежащего подчинённой клаузы;
  arg1_case='accs'        тег падежа дополнения подчинённой клаузы;
  main_subject            int, номер аргумента-подлежащего главной клаузы (по умолчанию дополнение подчинённой клаузы); 
  main_verb='нравиться',  глагол-предикат главной клаузы;
  arg0='Маша'             подлежащее подчинённой клаузы;
  arg1='Петя'             дополнение подчинённой клаузы;
  cap=True                капитализация аргументов-имен собственных.

  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.
  
  Example
  
  clause_wrapper('любить', 'Федя', 'Маша', 0)

  'Феде нравится, что Федя любит Машу'  
  '''
  if main_subject:
    inflected_main_subject = morph.parse(arg1)[0].inflect({'sing', 'datv'}).word.capitalize()
  else:
    inflected_main_subject = morph.parse(arg0)[0].inflect({'sing', 'datv'}).word
  if (cap == True) and ('Name' in morph.parse(inflected_main_subject)[0].tag):
    inflected_main_subject = inflected_main_subject.capitalize()


  inflected_main_verb = morph.parse(main_verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  subordinate_clause = a0_a1_clause_maker(verb, arg0, arg1, arg0_case, arg1_case, cap)
  main_clause = '{} {}'.format (inflected_main_subject, inflected_main_verb)
  sentence = '{}, что {}'.format(main_clause, subordinate_clause)

  return sentence

clause_wrapper('любить', 'Федя', 'Маша', 0)


'Феде нравится, что Федя любит Машу'

## **В процессе формирования сидовых множеств выяснилось, что косинусная дистанция очень сильно зависит от количества слов в сидовом множестве. Вот несколько примеров:**

In [0]:
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
#новостной корпус Тайги
!wget http://vectors.nlpl.eu/repository/20/187.zip
!unzip 187.zip

--2020-03-11 10:22:23--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz.1’

news_mystem_skipgra 100%[===================>] 524.47M  10.8MB/s    in 50s     

2020-03-11 10:23:14 (10.5 MB/s) - ‘news_mystem_skipgram_1000_20_2015.bin.gz.1’ saved [549952184/549952184]

--2020-03-11 10:23:16--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  9.75MB/s    in 9m 

In [5]:
skipgram_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
fasttext_model = gensim.models.KeyedVectors.load("model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def make_seed_embedding(seed_dict, model):
  '''
  Принимает на вход список, на выходе дает средний вектор всех слов из этого списка
  '''
  summ = 0
  if model == skipgram_model:
    for verb in seed_dict:
      summ = summ + model[verb + '_V']
    vector = summ/len(seed_dict)

# NB! Добавила берт
  # elif model == bc:
  #   for verb in seed_dict:
  #     summ = summ + bc.encode([verb])
  #   vector = summ/len(seed_dict)

  else:
    for verb in seed_dict:
      summ = summ + model.get_vector(verb)
    vector = summ/len(seed_dict)

  return vector

In [0]:
seed_1_dict = ['одобрять']
seed_2_dict = ['негодовать']
seed_3_dict = ['убивать']
seed_4_dict = ['хвалить']
seed_5_dict = ['убивать', 'негодовать']
seed_6_dict = ['хвалить', 'одобрять']

seed_1_skipgram = make_seed_embedding(seed_1_dict, skipgram_model)
seed_1_fasttext = make_seed_embedding(seed_1_dict, fasttext_model)

seed_2_skipgram = make_seed_embedding(seed_2_dict, skipgram_model)
seed_2_fasttext = make_seed_embedding(seed_2_dict, fasttext_model)

seed_3_skipgram = make_seed_embedding(seed_3_dict, skipgram_model)
seed_3_fasttext = make_seed_embedding(seed_3_dict, fasttext_model)

seed_4_skipgram = make_seed_embedding(seed_4_dict, skipgram_model)
seed_4_fasttext = make_seed_embedding(seed_4_dict, fasttext_model)

seed_5_skipgram = make_seed_embedding(seed_5_dict, skipgram_model)
seed_5_fasttext = make_seed_embedding(seed_5_dict, fasttext_model)


seed_6_skipgram = make_seed_embedding(seed_6_dict, skipgram_model)
seed_6_fasttext = make_seed_embedding(seed_6_dict, fasttext_model)

In [0]:
similarity = cosine_distances(np.atleast_2d(seed_1_skipgram), np.atleast_2d(seed_2_skipgram))
similarity2 = cosine_distances(np.atleast_2d(seed_3_skipgram), np.atleast_2d(seed_4_skipgram))
similarity3 = cosine_distances(np.atleast_2d(seed_5_skipgram), np.atleast_2d(seed_6_skipgram))
print(similarity, 'одобрять-негодовать')
print(similarity2, 'убивать-хвалить')
print(similarity3, 'одобрять, негодовать - убивать, хвалить')

In [0]:

def add_sim_list(model, my_verbs_list, seed_vector):
  '''
  Принимает на вход модель, словарь из глаголов, и средний вектор сидового множества, 
  возвращает словарь косинусных расстояний
  '''
  sim_list = []
  if model == skipgram_model:  
    for verb in my_verbs_list:
        try:
          similarity = cosine_distances(np.atleast_2d(model[verb + '_V']), np.atleast_2d(seed_vector))
        except KeyError as e:
          similarity = 'verb is not in model dict'
        sim_list.append(similarity)
  else:
    for verb in my_verbs_list:
      try:
        similarity = cosine_distances(np.atleast_2d(model.get_vector(verb)), np.atleast_2d(seed_vector))
      except (KeyError, AttributeError) as e:
        similarity = 'verb is not in model dict'
      sim_list.append(similarity)
  print(len(sim_list))
  print(len(my_verbs_list))

  #dataframe[df_pos] = sim_list
  return sim_list

In [0]:
def find_close_vector(model, my_verbs_list, seed_vector):
  

# Проверка расстояний до сидов

In [0]:
fnames = '''
neg_a0_a1_mutual.json  opp_a0_a1_candidates_l.json  pos_a1_a2_mutual.json
neg_a0_a2_mutual.json  pos_a0_a1_mutual.json	    sample_data
neg_a1_a2_mutual.json  pos_a0_a2_mutual.json
'''.split()

In [0]:
def verb_distance_df_w2v(verb_list, model,
                  seed1=['ценить'],
                  seed2=['убивать']):

  df = pd.DataFrame({'variant':verb_list})

  distance_to_seed1 = df['variant'].apply(
      lambda x: cosine_distances(
          np.atleast_2d(make_seed_embedding([x], model)), 
          np.atleast_2d(make_seed_embedding(seed1, model))).item() 
          if (x+'_V' in model.vocab) or (x in model.vocab) else None)
  distance_to_seed2 = df['variant'].apply(
      lambda x: cosine_distances(
          np.atleast_2d(make_seed_embedding([x], model)), 
          np.atleast_2d(make_seed_embedding(seed2, model))).item() 
          if (x+'_V' in model.vocab) or (x in model.vocab) else None)

  df['distance_to_seed1'] = distance_to_seed1
  df['distance_to_seed2'] = distance_to_seed2

  df['attributed_polarity'] = np.where(
      # если дистанция до первого сида меньше
      df['distance_to_seed1'] <= df['distance_to_seed2'], 
      # то в результирующую колонку кладём тру (иначе - фолс)
      True, False)
  
  df['seed1'] = ' '.join(seed1)
  df['seed2'] = ' '.join(seed2)

  return df

In [0]:
def highlight_true(s):
    '''
    green
    '''
    is_true = s == True
    return ['background-color: green' if v else '' for v in is_true]

def highlight_none(s):
    '''
    grey
    '''
    is_none = s == None
    return ['background-color: grey' if v else '' for v in is_none]

**Skipgram**

In [32]:
  list2check = ['порицать', 'осуждать', 'негодовать',
            'обвинять', 'наказывать', 
            'ненавидеть', 'убивать', 'разрушать', 'одобрять', 'хвалить', 'поощрять', 
            'обожать', 'восхищаться', 'восторгаться', 
            'нравиться', 'ценить', 'гордиться',
            'хвалить', 'нахваливать', 'превозносить', 'фанфарониться']
df = verb_distance_df_w2v(list2check, skipgram_model)
df

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,порицать,0.858294,0.825797,False,ценить,убивать
1,осуждать,0.868024,0.722466,False,ценить,убивать
2,негодовать,0.797929,0.805689,True,ценить,убивать
3,обвинять,0.894837,0.753185,False,ценить,убивать
4,наказывать,0.901647,0.813960,False,ценить,убивать
5,ненавидеть,0.716300,0.695951,False,ценить,убивать
6,убивать,0.950652,0.000000,False,ценить,убивать
7,разрушать,0.901834,0.686278,False,ценить,убивать
8,одобрять,0.907819,0.915080,True,ценить,убивать
9,хвалить,0.644141,0.853935,True,ценить,убивать


In [33]:
styled = df.style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,порицать,0.858294,0.825797,False,ценить,убивать
1,осуждать,0.868024,0.722466,False,ценить,убивать
2,негодовать,0.797929,0.805689,True,ценить,убивать
3,обвинять,0.894837,0.753185,False,ценить,убивать
4,наказывать,0.901647,0.81396,False,ценить,убивать
5,ненавидеть,0.7163,0.695951,False,ценить,убивать
6,убивать,0.950652,0,False,ценить,убивать
7,разрушать,0.901834,0.686278,False,ценить,убивать
8,одобрять,0.907819,0.91508,True,ценить,убивать
9,хвалить,0.644141,0.853935,True,ценить,убивать


Fasttext

In [0]:
# with open("neg_a0_a1_mutual.json", "r", encoding="utf-8") as f:
  # list2check = json.load(f, encoding="utf-8")
  list2check = ['порицать', 'осуждать', 'негодовать',
            'обвинять', 'наказывать', 
            'ненавидеть', 'убивать', 'разрушать', 'одобрять', 'хвалить', 'поощрять', 
            'обожать', 'восхищаться', 'восторгаться', 
            'нравиться', 'ценить', 'гордиться',
            'хвалить', 'нахваливать', 'превозносить', 'фанфарониться']
df = verb_distance_df_w2v(list2check, fasttext_model)
df

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,порицать,4.828671e-01,5.608901e-01,True,ценить,убивать
1,осуждать,5.432622e-01,4.932728e-01,False,ценить,убивать
2,негодовать,5.366783e-01,4.292440e-01,False,ценить,убивать
3,обвинять,6.626669e-01,4.549050e-01,False,ценить,убивать
4,наказывать,5.197760e-01,3.627480e-01,False,ценить,убивать
5,ненавидеть,5.238245e-01,4.368629e-01,False,ценить,убивать
6,убивать,6.468743e-01,2.980232e-07,False,ценить,убивать
7,разрушать,6.433487e-01,4.372949e-01,False,ценить,убивать
8,одобрять,4.799517e-01,6.122896e-01,True,ценить,убивать
9,хвалить,4.394785e-01,6.178040e-01,True,ценить,убивать


Странные комбинации отношений

In [0]:
with open("opp_a0_a1_candidates_l.json", "r", encoding="utf-8") as f:
  list2check = json.load(f, encoding="utf-8")

In [0]:
df = verb_distance_df_w2v(list2check, skipgram_model)

In [41]:
styled = df[df['distance_to_seed1'].notnull()].style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
1,вымаливать,0.86026,0.84947,False,ценить,убивать
3,молить,0.840595,0.780412,False,ценить,убивать
4,умолять,0.819918,0.777008,False,ценить,убивать
5,бахвалиться,0.842104,0.79987,False,ценить,убивать
6,козырять,0.815601,0.927647,True,ценить,убивать
8,похвалиться,0.840586,0.896548,True,ценить,убивать
10,похвастаться,0.82666,0.888687,True,ценить,убивать
14,хвалиться,0.793424,0.839402,True,ценить,убивать
15,хвастаться,0.803414,0.82146,True,ценить,убивать


In [42]:
df = verb_distance_df_w2v(list2check, fasttext_model)
df

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,взмолиться,0.783267,0.559065,False,ценить,убивать
1,вымаливать,0.598771,0.449866,False,ценить,убивать
2,вымолить,0.702251,0.495651,False,ценить,убивать
3,молить,0.748107,0.589865,False,ценить,убивать
4,умолять,0.718002,0.563480,False,ценить,убивать
5,бахвалиться,0.545554,0.603213,True,ценить,убивать
6,козырять,0.672148,0.616114,False,ценить,убивать
7,козырнуть,0.898142,0.707615,False,ценить,убивать
8,похвалиться,0.501033,0.667736,True,ценить,убивать
9,похваляться,0.409907,0.595190,True,ценить,убивать


In [43]:
styled = df[df['distance_to_seed1'].notnull()].style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,взмолиться,0.783267,0.559065,False,ценить,убивать
1,вымаливать,0.598771,0.449866,False,ценить,убивать
2,вымолить,0.702251,0.495651,False,ценить,убивать
3,молить,0.748107,0.589865,False,ценить,убивать
4,умолять,0.718002,0.56348,False,ценить,убивать
5,бахвалиться,0.545554,0.603213,True,ценить,убивать
6,козырять,0.672148,0.616114,False,ценить,убивать
7,козырнуть,0.898142,0.707615,False,ценить,убивать
8,похвалиться,0.501033,0.667736,True,ценить,убивать
9,похваляться,0.409907,0.59519,True,ценить,убивать


In [0]:
with open("/content/neg_a0_a1_mutual.json", "r", encoding="utf-8") as f:

  list2check = json.load(f, encoding="utf-8")

In [46]:
df = verb_distance_df_w2v(list2check, skipgram_model)
styled = df[df['distance_to_seed1'].notnull()].style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,бояться,0.830213,0.801919,False,ценить,убивать
1,запаниковать,0.89024,0.83246,False,ценить,убивать
2,испугаться,0.885396,0.725463,False,ценить,убивать
3,напугаться,0.916262,0.871984,False,ценить,убивать
4,опасаться,0.892114,0.816198,False,ценить,убивать
6,паниковать,0.850504,0.862941,True,ценить,убивать
7,перепугаться,0.866432,0.81828,False,ценить,убивать
8,побаиваться,0.803369,0.828339,True,ценить,убивать
9,побояться,0.829957,0.784962,False,ценить,убивать
10,пугаться,0.844846,0.829192,False,ценить,убивать


In [47]:
df = verb_distance_df_w2v(list2check, fasttext_model)
styled = df[df['distance_to_seed1'].notnull()].style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,бояться,0.679239,0.572352,False,ценить,убивать
1,запаниковать,0.569074,0.495499,False,ценить,убивать
2,испугаться,0.785484,0.526113,False,ценить,убивать
3,напугаться,0.799248,0.498026,False,ценить,убивать
4,опасаться,0.660627,0.479723,False,ценить,убивать
6,паниковать,0.536416,0.421515,False,ценить,убивать
7,перепугаться,0.857093,0.561067,False,ценить,убивать
8,побаиваться,0.535853,0.57072,True,ценить,убивать
9,побояться,0.714334,0.559884,False,ценить,убивать
10,пугаться,0.713686,0.534152,False,ценить,убивать


In [0]:
with open("/content/pos_a0_a1_mutual.json", "r", encoding="utf-8") as f:

  list2check = json.load(f, encoding="utf-8")

In [49]:
df = verb_distance_df_w2v(list2check, skipgram_model)
styled = df[df['distance_to_seed1'].notnull()].style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,ассистировать,0.873889,0.89396,True,ценить,убивать
2,вызволять,0.906174,0.767849,False,ценить,убивать
3,выручать,0.877507,0.901189,True,ценить,убивать
5,курировать,0.92051,0.917018,False,ценить,убивать
6,патронировать,0.883823,0.886602,True,ценить,убивать
9,помогать,0.747361,0.835255,True,ценить,убивать
13,поддерживать,0.796734,0.827589,True,ценить,убивать
14,подсоблять,0.913005,0.88328,False,ценить,убивать
16,подстраховывать,0.863129,0.894918,True,ценить,убивать
17,посодействовать,0.890899,0.871728,False,ценить,убивать


In [50]:
df = verb_distance_df_w2v(list2check, fasttext_model)
styled = df[df['distance_to_seed1'].notnull()].style.apply(highlight_true)
styled

,variant,distance_to_seed1,distance_to_seed2,attributed_polarity,seed1,seed2
0,ассистировать,0.520183,0.581076,True,ценить,убивать
1,вызволить,0.77915,0.464746,False,ценить,убивать
2,вызволять,0.665571,0.493201,False,ценить,убивать
3,выручать,0.468548,0.462247,False,ценить,убивать
4,выручить,0.513366,0.485034,False,ценить,убивать
5,курировать,0.548345,0.629481,True,ценить,убивать
6,патронировать,0.587306,0.57833,False,ценить,убивать
7,поддержать,0.474653,0.548624,True,ценить,убивать
8,подсобить,0.729468,0.56586,False,ценить,убивать
9,помогать,0.580591,0.563997,False,ценить,убивать
